In [ ]:
# pip install ollama pandas openpyxl
# !pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [ ]:
import os
import re
from typing import Dict, List, Optional
import pandas as pd
import data_extraction

import json
import random
from difflib import get_close_matches

In [2]:
raw_data = data_extraction.extract_all("data")

In [3]:
columns = ["project_title", "client", "project_type"]
df_cleaned = raw_data[columns].copy()
df_cleaned["project_title_and_client"] = df_cleaned['project_title'] + " " + df_cleaned['client']
df_cleaned

,project_title,client,project_type,project_title_and_client
0,132027 - Moorabbin Hospital - Carpark Conver,AUVICKAN - Kane Construction Pty Ltd,Community & Institutional,132027 - Moorabbin Hospital - Carpark Conver A...
1,130348 - New Epping - Medical Building,AUVICRIVE - Riverlee Pty Ltd,Community & Institutional,130348 - New Epping - Medical Building AUVICRI...
2,130524 - RWH Rapid Access Hub,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,130524 - RWH Rapid Access Hub GRAY981 - GRAY P...
3,131450 - Monash Medical Centre Tower Exp,AUVICLYO - Lyons Architects,Community & Institutional,131450 - Monash Medical Centre Tower Exp AUVIC...
4,131568 - RWH Level 3 Birthing Suite,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,131568 - RWH Level 3 Birthing Suite GRAY981 - ...
...,...,...,...,...
17058,EKU Energy,Fortec,Industrial,EKU Energy Fortec
17059,STT (Peer review)_PO#1,STT,Industrial,STT (Peer review)_PO#1 STT
17060,Air Trunk (MEP),Air Trunk,Industrial,Air Trunk (MEP) Air Trunk
17061,VDC KIX12,Taisei Corporation,Industrial,VDC KIX12 Taisei Corporation


In [4]:
df_cleaned = df_cleaned[~df_cleaned["project_type"].isin([None, "nan", "Cancelled"])]

project_type_map = {
    "commercial": "Commercial",
    "Community & Instituitional": "Community & Institutional",
}

df_cleaned["project_type"] = df_cleaned["project_type"].replace(project_type_map)

df_cleaned

C:\Users\jeremy.yap\AppData\Local\Temp\ipykernel_70288\3057930727.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["project_type"] = df_cleaned["project_type"].replace(project_type_map)


,project_title,client,project_type,project_title_and_client
0,132027 - Moorabbin Hospital - Carpark Conver,AUVICKAN - Kane Construction Pty Ltd,Community & Institutional,132027 - Moorabbin Hospital - Carpark Conver A...
1,130348 - New Epping - Medical Building,AUVICRIVE - Riverlee Pty Ltd,Community & Institutional,130348 - New Epping - Medical Building AUVICRI...
2,130524 - RWH Rapid Access Hub,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,130524 - RWH Rapid Access Hub GRAY981 - GRAY P...
3,131450 - Monash Medical Centre Tower Exp,AUVICLYO - Lyons Architects,Community & Institutional,131450 - Monash Medical Centre Tower Exp AUVIC...
4,131568 - RWH Level 3 Birthing Suite,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,131568 - RWH Level 3 Birthing Suite GRAY981 - ...
...,...,...,...,...
17058,EKU Energy,Fortec,Industrial,EKU Energy Fortec
17059,STT (Peer review)_PO#1,STT,Industrial,STT (Peer review)_PO#1 STT
17060,Air Trunk (MEP),Air Trunk,Industrial,Air Trunk (MEP) Air Trunk
17061,VDC KIX12,Taisei Corporation,Industrial,VDC KIX12 Taisei Corporation


In [5]:
df_cleaned["project_type"].value_counts()

project_type
Commercial                   5942
Industrial                   2244
Residential                  2162
Community & Institutional    2131
Infrastructure               1611
Hotel & Resort               1371
Mixed Use Development         750
Water & Environment           253
Aviation                      215
Healthcare                    105
Data Center                    88
Mining & Resources             87
residential                     5
Institutional                   1
Name: count, dtype: int64

In [6]:
unique_categories = df_cleaned['project_type'].unique().tolist()
unique_categories = [e for e in unique_categories if e not in (None, "nan", "Cancelled", "Institutional")]
unique_categories

labels = sorted(unique_categories)
labels

['Aviation',
 'Commercial',
 'Community & Institutional',
 'Data Center',
 'Healthcare',
 'Hotel & Resort',
 'Industrial',
 'Infrastructure',
 'Mining & Resources',
 'Mixed Use Development',
 'Residential',
 'Water & Environment',
 'residential']

In [7]:
df_final = df_cleaned[df_cleaned["project_type"]!="Institutional"]
df_final

,project_title,client,project_type,project_title_and_client
0,132027 - Moorabbin Hospital - Carpark Conver,AUVICKAN - Kane Construction Pty Ltd,Community & Institutional,132027 - Moorabbin Hospital - Carpark Conver A...
1,130348 - New Epping - Medical Building,AUVICRIVE - Riverlee Pty Ltd,Community & Institutional,130348 - New Epping - Medical Building AUVICRI...
2,130524 - RWH Rapid Access Hub,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,130524 - RWH Rapid Access Hub GRAY981 - GRAY P...
3,131450 - Monash Medical Centre Tower Exp,AUVICLYO - Lyons Architects,Community & Institutional,131450 - Monash Medical Centre Tower Exp AUVIC...
4,131568 - RWH Level 3 Birthing Suite,GRAY981 - GRAY PUKSAND PTY LTD,Community & Institutional,131568 - RWH Level 3 Birthing Suite GRAY981 - ...
...,...,...,...,...
17058,EKU Energy,Fortec,Industrial,EKU Energy Fortec
17059,STT (Peer review)_PO#1,STT,Industrial,STT (Peer review)_PO#1 STT
17060,Air Trunk (MEP),Air Trunk,Industrial,Air Trunk (MEP) Air Trunk
17061,VDC KIX12,Taisei Corporation,Industrial,VDC KIX12 Taisei Corporation


In [9]:
df_final.to_parquet("data/final_data.parquet", index=False)

## NLP Classifier - HuggingFace

In [10]:
import transformers, accelerate
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)

transformers: 4.56.0
accelerate: 1.10.1


In [11]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scikit-learn datasets transformers evaluate

Note: you may need to restart the kernel to use updated packages.


In [14]:
df_final = pd.read_parquet("data/final_data.parquet")

In [ ]:
# ====== CONFIG ======
MODEL_NAME = "distilbert-base-uncased"      # try 'distilbert-base-multilingual-cased' if non-English
MAX_LENGTH = 64                              # titles are short; 64–128 is plenty
BATCH_SIZE = 16                              # raise if GPU has headroom
LR = 3e-5
EPOCHS = 3
SEED = 42
OUTPUT_DIR = "distilbert_projects"

# ====== 1) Load & clean ======
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assumes you already have df with these two columns:
# df['project_title_and_client'], df['project_type']

# Normalize labels (fix case variants like 'Commercial' vs 'commercial')
df_final["project_type"] = (
    df_final["project_type"]
    .astype(str)
    .str.strip()
    .replace({"nan": np.nan, "": np.nan})
    .str.replace("&", "and", regex=False)
    .str.title()   # 'community & institutional' -> 'Community And Institutional'
)

# ====== 2) Encode labels & stratified split ======
le = LabelEncoder()
df_final["labels"] = le.fit_transform(df_final["project_type"])
id2label = {i: l for i, l in enumerate(le.classes_)}
label2id = {l: i for i, l in id2label.items()}
num_labels = len(id2label)
print("Labels:", label2id)

X_train, X_val = train_test_split(
    df_final,
    test_size=0.3,
    random_state=SEED,
    stratify=df_final["labels"]
)

# ====== 3) Build HF Dataset & tokenize ======
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess(batch):
    return tokenizer(
        batch["project_title_and_client"],
        truncation=True,
        max_length=MAX_LENGTH
    )

train_ds = Dataset.from_pandas(X_train[["project_title_and_client", "labels"]])
val_ds   = Dataset.from_pandas(X_val[["project_title_and_client", "labels"]])

train_ds = train_ds.map(preprocess, batched=True, remove_columns=["project_title_and_client"])
val_ds   = val_ds.map(preprocess,   batched=True, remove_columns=["project_title_and_client"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ====== 4) Define model & Trainer ======
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import confusion_matrix

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"]
        # "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,                             # uses mixed precision if GPU+AMP available
    logging_steps=50,
    seed=SEED,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# ====== 5) Train ======
trainer.train()

# ====== 6) Evaluate (accuracy, F1, confusion matrix) ======
eval_metrics = trainer.evaluate()
print("\nEval metrics:", eval_metrics)

# Confusion matrix (nice to inspect per-class behavior)
pred_logits = trainer.predict(val_ds).predictions
y_pred = pred_logits.argmax(axis=1)
y_true = np.array(X_val["labels"])

cm = confusion_matrix(y_true, y_pred, labels=range(num_labels))
cm_df = pd.DataFrame(cm, index=[f"true:{id2label[i]}" for i in range(num_labels)],
                        columns=[f"pred:{id2label[i]}" for i in range(num_labels)])
# print("\nConfusion matrix:\n", cm_df)

# ====== 7) Save model & tokenizer ======
trainer.save_model(OUTPUT_DIR)            # saves the fine-tuned weights
tokenizer.save_pretrained(OUTPUT_DIR)

# ====== 8) Inference helper on new text ======
def predict_label(text: str) -> str:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
    with torch.no_grad():
        logits = model(**inputs).logits
    return id2label[int(logits.argmax(dim=-1).item())]


Labels: {'Aviation': 0, 'Commercial': 1, 'Community And Institutional': 2, 'Data Center': 3, 'Healthcare': 4, 'Hotel And Resort': 5, 'Industrial': 6, 'Infrastructure': 7, 'Mining And Resources': 8, 'Mixed Use Development': 9, 'Residential': 10, 'Water And Environment': 11}


Map: 100%|██████████| 5090/5090 [00:00<00:00, 38148.98 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jeremy.yap\AppData\Local\Temp\ipykernel_104992\3114021289.py:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Accuracy
200,1.326600,1.221702,0.635953
400,1.027700,0.947243,0.706680
600,0.808800,0.780011,0.766405
800,0.596100,0.683392,0.793713
1000,0.482800,0.620086,0.823379
1200,0.441400,0.552149,0.844008
1400,0.420700,0.483984,0.863654
1600,0.302200,0.439542,0.879568
1800,0.268800,0.437443,0.882318
2000,0.301300,0.415840,0.891552


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' 


Eval metrics: {'eval_loss': 0.40662965178489685, 'eval_accuracy': 0.8937131630648331, 'eval_runtime': 61.6406, 'eval_samples_per_second': 82.575, 'eval_steps_per_second': 5.175, 'epoch': 3.0}


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\NLP Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Confusion matrix:
                                   pred:Aviation  pred:Commercial  \
true:Aviation                                62                2   
true:Commercial                               1             1614   
true:Community And Institutional              3               17   
true:Data Center                              0                3   
true:Healthcare                               0                1   
true:Hotel And Resort                         0               18   
true:Industrial                               1               18   
true:Infrastructure                           1               17   
true:Mining And Resources                     0                0   
true:Mixed Use Development                    2               15   
true:Residential                              0               28   
true:Water And Environment                    0                1   

                                  pred:Community And Institutional  \
true:Aviation            

In [1]:
cm_df

NameError: name 'cm_df' is not defined